Import Stuff

In [23]:
import ssl
import re


from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from umap import UMAP

from sklearn.datasets import fetch_20newsgroups
from sklearn.cluster import KMeans

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from datetime import datetime
from textblob import TextBlob
from textblob_de import TextBlobDE
from datasets import load_dataset

import pickle
import importlib
import functions
import requests





In [3]:
from functions import *
importlib.reload(functions)

nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

parties = ["CDU", "SPD", "FDP", "AFD", "LEFT", "GREENS"]
file_path = "C:/Users/Jacob/OneDrive/uni/MA WiSoz/Semester III/Computational Social Sciences/foPra/data/"


# Open the file in read mode
with open("API Keys/DeepL.txt", 'r', encoding='utf-8') as file:
    # Read the contents of the file
    api_key_deepl = file.read()

url = 'https://api-free.deepl.com/v2/translate'


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Edit Text documents

In [4]:
df_manifesto = pd.DataFrame()
for p in parties:
    new_df = pd.read_csv(file_path + "parties/"+ p + "_manifesto.csv")
    new_df["party"] = p  # Add a new column for the party name
    df_manifesto = pd.concat([df_manifesto, new_df], axis=0)


df_manifesto['date'] = pd.to_datetime(df_manifesto['date'], format='%Y%m')



In [5]:
split_date = datetime(1998, 9, 1)
df_manifesto = split_text_by_date(df_manifesto, split_date)

# Count occurrences of distinct values in the party column
party_counts = df_manifesto["party"].value_counts()

# Add a line that contains this count per party
df_manifesto["party_count"] = df_manifesto["party"].map(party_counts)
df_manifesto["cmp_code"] = pd.to_numeric(df_manifesto["cmp_code"], errors='coerce')
df_manifesto.index = range(1, len(df_manifesto) + 1)

# Delete entries that do not contain words in the text column
df_manifesto = df_manifesto[df_manifesto['text'].str.contains(r'\b\w+\b', na=False)]

# Ensure the 'text' column contains only strings and handle NaN values
df_manifesto["text"] = df_manifesto["text"].astype(str).fillna("")

df_manifesto_final = merge_text_rows(df_manifesto)

In [29]:
df_manifesto_final["sentiment"] = df_manifesto_final["text"].apply(lambda x: TextBlobDE(x).sentiment.polarity)



In [31]:
df_manifesto_final["date"] = df_manifesto_final["date"].dt.year

In [28]:
test = TextBlobDE(df_manifesto_final["text"][2000])
test.sentiment.polarity


0.7999999999999999

In [32]:
df_manifesto_final.to_csv("data/parties/All_Manifestos.csv", index=True)